In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Input, Concatenate, Dropout, Reshape
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.optimizers import Adam
import pickle
import warnings
warnings.filterwarnings('ignore')

2025-05-26 12:20:51.107397: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-26 12:20:51.119310: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748272851.133983  150162 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748272851.138608  150162 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748272851.149263  150162 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

## Função listar_usuarios

In [7]:
def listar_usuarios(data_path):
    """
    retornar todos os usuários do arquivo
    
    Args:
        data_path: Caminho para o arquivo CSV
    
    Returns:
        list: Nomes de todos os usuários do arquivo
    """
    print("=== OBTENDO USUÁRIOS DO ARQUIVO ===")
    
    # Carregar dados
    df = pd.read_csv(
        data_path, 
        sep=';', 
        encoding='utf-8', 
        parse_dates=['DataHoraCriacao'], 
        dayfirst=True
    )
    
    return df['usuario'].unique()

## Função analisar_distribuicao_classes

In [9]:
def analisar_distribuicao_classes(serie_y, username):
    """
    Analisa a distribuição das classes para o usuário
    
    Args:
        serie_y: Série com os targets
        username: Nome do usuário
    
    Returns:
        dict: Estatísticas das classes
    """
    print(f"\n=== ANÁLISE DE CLASSES PARA {username} ===")
    
    class_counts = serie_y.value_counts()
    print(f"Total de classes: {len(class_counts)}")
    print(f"Total de amostras: {len(serie_y)}")
    print(f"Média de amostras por classe: {class_counts.mean():.1f}")
    print(f"Mediana de amostras por classe: {class_counts.median():.1f}")
    
    # Classes com poucas amostras
    classes_com_1_amostra = class_counts[class_counts == 1]
    classes_com_2_5_amostras = class_counts[(class_counts >= 2) & (class_counts <= 5)]
    
    print(f"\nClasses com 1 amostra: {len(classes_com_1_amostra)}")
    print(f"Classes com 2-5 amostras: {len(classes_com_2_5_amostras)}")
    print(f"Classes com 6+ amostras: {len(class_counts[class_counts > 5])}")
    
    if len(classes_com_1_amostra) > 0:
        print(f"\n⚠️ Classes problemáticas (1 amostra):")
        print(classes_com_1_amostra.head(10))
    
    # Top classes
    print(f"\n📊 Top 10 classes mais frequentes:")
    print(class_counts.head(10))
    
    return {
        'total_classes': len(class_counts),
        'total_samples': len(serie_y),
        'classes_with_one_sample': len(classes_com_1_amostra),
        'class_counts': class_counts
    }

## Função recomendar_usuarios

Analisa a quantidade de amostras e de classes(casos de uso) envolvidas 

In [10]:
def recomendar_usuarios(data_path, min_amostras=50, min_classes=5):
    """
    Analisa todos os usuários e recomenda os melhores para treinamento
    
    Args:
        data_path: Caminho para o arquivo CSV
        min_amostras: Número mínimo de amostras por usuário
        min_classes: Número mínimo de classes diferentes por usuário
    
    Returns:
        list: Lista de usuários recomendados ordenados por qualidade
    """
    print("=== ANALISANDO USUÁRIOS PARA RECOMENDAÇÃO ===")
    
    # Carregar dados
    df = pd.read_csv(
        data_path, 
        sep=';', 
        encoding='utf-8', 
        parse_dates=['DataHoraCriacao'], 
        dayfirst=True
    )
    
    usuarios_stats = []
    
    for usuario in df['usuario'].unique():
        df_user = df[df['usuario'] == usuario]
        class_counts = df_user['casoDeUso'].value_counts()
        
        # Calcular métricas de qualidade
        total_amostras = len(df_user)
        total_classes = len(class_counts)
        classes_com_uma_amostra = len(class_counts[class_counts == 1])
        classes_com_multiplas_amostras = len(class_counts[class_counts > 1])
        media_amostras_por_classe = class_counts.mean()
        balanceamento = 1 - (class_counts.std() / class_counts.mean()) if class_counts.mean() > 0 else 0
        
        # Score de qualidade (0-1, onde 1 é melhor)
        score_amostras = min(total_amostras / 200, 1.0)  # Normaliza até 200 amostras
        score_classes = min(total_classes / 20, 1.0)  # Normaliza até 20 classes
        score_balanceamento = max(0, balanceamento)  # Evita valores negativos
        score_sem_singletons = classes_com_multiplas_amostras / total_classes if total_classes > 0 else 0
        
        # Score final ponderado
        score_final = (
            0.3 * score_amostras + 
            0.3 * score_classes + 
            0.2 * score_balanceamento + 
            0.2 * score_sem_singletons
        )
        
        usuarios_stats.append({
            'usuario': usuario,
            'total_amostras': total_amostras,
            'total_classes': total_classes,
            'classes_com_uma_amostra': classes_com_uma_amostra,
            'classes_com_multiplas_amostras': classes_com_multiplas_amostras,
            'media_amostras_por_classe': media_amostras_por_classe,
            'balanceamento': balanceamento,
            'score_final': score_final
        })
    
    # Ordenar por score final
    usuarios_stats.sort(key=lambda x: x['score_final'], reverse=True)
    
    # Filtrar usuários que atendem critérios mínimos
    usuarios_validos = [
        u for u in usuarios_stats 
        if u['total_amostras'] >= min_amostras and u['total_classes'] >= min_classes
    ]
    
    print(f"\n📊 Top 10 usuários recomendados:")
    print("-" * 100)
    print(f"{'Usuário':<12} {'Amostras':<9} {'Classes':<8} {'Singleton':<10} {'Score':<8} {'Qualidade'}")
    print("-" * 100)
    
    for i, user_stats in enumerate(usuarios_validos[:10]):
        qualidade = "Excelente" if user_stats['score_final'] > 0.7 else "Boa" if user_stats['score_final'] > 0.5 else "Regular"
        print(f"{user_stats['usuario']:<12} {user_stats['total_amostras']:<9} {user_stats['total_classes']:<8} "
              f"{user_stats['classes_com_uma_amostra']:<10} {user_stats['score_final']:.3f}     {qualidade}")
    
    if not usuarios_validos:
        print("⚠️ Nenhum usuário atende os critérios mínimos especificados.")
        return usuarios_stats[:5]  # Retorna os 5 melhores mesmo que não atendam critérios
    
    return usuarios_validos


In [11]:
recomendar_usuarios(data_path='../dados/processados/Dados_TechChallenge_Fase3.csv')

=== ANALISANDO USUÁRIOS PARA RECOMENDAÇÃO ===

📊 Top 10 usuários recomendados:
----------------------------------------------------------------------------------------------------
Usuário      Amostras  Classes  Singleton  Score    Qualidade
----------------------------------------------------------------------------------------------------
usuario_04   2666      76       2          0.795     Excelente
usuario_03   21832     189      10         0.789     Excelente
usuario_05   15783     166      13         0.784     Excelente
usuario_07   8189      180      16         0.782     Excelente
usuario_14   13695     135      15         0.778     Excelente
usuario_15   24020     190      22         0.777     Excelente
usuario_00   717       69       8          0.777     Excelente
usuario_10   20070     186      25         0.773     Excelente
usuario_08   3291      21       3          0.771     Excelente
usuario_09   2497      126      24         0.762     Excelente


[{'usuario': 'usuario_04',
  'total_amostras': 2666,
  'total_classes': 76,
  'classes_com_uma_amostra': 2,
  'classes_com_multiplas_amostras': 74,
  'media_amostras_por_classe': np.float64(35.078947368421055),
  'balanceamento': np.float64(-0.6061146506119008),
  'score_final': 0.7947368421052632},
 {'usuario': 'usuario_03',
  'total_amostras': 21832,
  'total_classes': 189,
  'classes_com_uma_amostra': 10,
  'classes_com_multiplas_amostras': 179,
  'media_amostras_por_classe': np.float64(115.51322751322752),
  'balanceamento': np.float64(-1.729370023793121),
  'score_final': 0.7894179894179894},
 {'usuario': 'usuario_05',
  'total_amostras': 15783,
  'total_classes': 166,
  'classes_com_uma_amostra': 13,
  'classes_com_multiplas_amostras': 153,
  'media_amostras_por_classe': np.float64(95.07831325301204),
  'balanceamento': np.float64(-1.6502169926387826),
  'score_final': 0.7843373493975904},
 {'usuario': 'usuario_07',
  'total_amostras': 8189,
  'total_classes': 180,
  'classes_com

## Comparando treino por usuário

Realiza o treino para cada usuário e compara qual usuário tem melhores condições de treino.

In [12]:
print("🚀 Iniciando pipeline de Machine Learning por usuário...")
print("=" * 60)

data_path='../dados/processados/Dados_TechChallenge_Fase3.csv'

# Testar Listar usuários
lista_usuarios = listar_usuarios(data_path=data_path)
print(f"Usuários encontrados: {lista_usuarios}")
print("=" * 60)

# Dicionário para armazenar todos os resultados
resultados_usuarios = {}

# Loop através de todos os usuários
for i, usuario in enumerate(lista_usuarios, 1):
    print(f"\n🔄 Processando usuário {i}/{len(lista_usuarios)}: {usuario}")
    print("-" * 40)
    
    try:
        model, history, results = main(
            data_path=data_path,
            usuario=usuario,
            use_lstm=False,  # Usar Dense layers (mais estável)
            epochs=50,
            plotar_resultado=False,
            salvar_modelo=False
        )
        
        # Debug: verificar estrutura dos resultados
        print(f"Debug - Tipo de 'results': {type(results)}")
        print(f"Debug - Chaves disponíveis: {list(results.keys()) if isinstance(results, dict) else 'N/A'}")
        
        # Armazenar resultados do usuário
        resultados_usuarios[usuario] = {
            'model': model,
            'history': history,
            'results': results,
            'status': 'sucesso'
        }
        
        print(f"✅ Usuário {usuario} processado com sucesso!")
        
    except Exception as e:
        print(f"❌ Erro ao processar usuário {usuario}: {str(e)}")
        resultados_usuarios[usuario] = {
            'model': None,
            'history': None,
            'results': None,
            'status': 'erro',
            'erro': str(e)
        }

print("\n" + "=" * 60)
print("COMPARAÇÃO DE RESULTADOS")
print("=" * 60)

# Comparar resultados
usuarios_sucesso = []
usuarios_erro = []

for usuario, dados in resultados_usuarios.items():
    if dados['status'] == 'sucesso':
        usuarios_sucesso.append(usuario)
        results = dados['results']
        
        # Extrair métricas de classificação
        accuracy = results.get('accuracy', 'N/A')
        confusion_matrix = results.get('confusion_matrix', 'N/A')
        class_names = results.get('class_names', 'N/A')
        
        # Calcular métricas adicionais da matriz de confusão
        precision = recall = f1_score = 'N/A'
        
        if isinstance(confusion_matrix, type(results.get('confusion_matrix'))) and hasattr(confusion_matrix, 'shape'):
            try:
                # Para classificação binária
                tn, fp, fn, tp = confusion_matrix.ravel()
                precision = tp / (tp + fp) if (tp + fp) > 0 else 0
                recall = tp / (tp + fn) if (tp + fn) > 0 else 0
                f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
            except:
                pass
        
        print(f"\nUsuário: {usuario}")
        print(f"Accuracy: {accuracy:.4f}" if isinstance(accuracy, (int, float)) else f"Accuracy: {accuracy}")
        print(f"Precision: {precision:.4f}" if isinstance(precision, (int, float)) else f"Precision: {precision}")
        print(f"Recall: {recall:.4f}" if isinstance(recall, (int, float)) else f"Recall: {recall}")
        print(f"F1-Score: {f1_score:.4f}" if isinstance(f1_score, (int, float)) else f"F1-Score: {f1_score}")
        print(f"Classes: {class_names}")
    else:
        usuarios_erro.append(usuario)
        print(f"\n❌ Usuário: {usuario} - ERRO: {dados['erro']}")

# Encontrar o melhor usuário baseado em R²
if usuarios_sucesso:
    print("\n" + "=" * 60)
    print("🏆 RANKING DOS MELHORES RESULTADOS")
    print("=" * 60)
    
    # Criar lista de usuários com suas métricas para ranking
    usuarios_com_metricas = []
    for usuario in usuarios_sucesso:
        results = resultados_usuarios[usuario]['results']
        
        # Usar accuracy como métrica principal para ranking
        accuracy_score = results.get('accuracy', 0)
        
        if isinstance(accuracy_score, (int, float)):
            usuarios_com_metricas.append((usuario, accuracy_score, results))
    
    # Ordenar por Accuracy (maior é melhor)
    usuarios_com_metricas.sort(key=lambda x: x[1], reverse=True)
    
    print(f"\n🥇 TOP 3 USUÁRIOS:")
    for i, (usuario, accuracy_score, results) in enumerate(usuarios_com_metricas[:3], 1):
        
        # Calcular métricas adicionais
        confusion_matrix = results.get('confusion_matrix')
        precision = recall = f1_score = 'N/A'
        
        if hasattr(confusion_matrix, 'ravel'):
            try:
                tn, fp, fn, tp = confusion_matrix.ravel()
                precision = tp / (tp + fp) if (tp + fp) > 0 else 0
                recall = tp / (tp + fn) if (tp + fn) > 0 else 0
                f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
            except:
                pass
        
        print(f"{i}º lugar - {usuario}")
        print(f"Accuracy: {accuracy_score:.4f}")
        print(f"Precision: {precision:.4f}" if isinstance(precision, (int, float)) else f"Precision: {precision}")
        print(f"F1-Score: {f1_score:.4f}" if isinstance(f1_score, (int, float)) else f"F1-Score: {f1_score}")
    
    melhor_usuario = usuarios_com_metricas[0][0]
    print(f"\n🎯 MELHOR MODELO: {melhor_usuario} (Accuracy: {usuarios_com_metricas[0][1]:.4f})")
    
# Resumo estatístico
print("\n" + "=" * 60)
print("RESUMO ESTATÍSTICO")
print("=" * 60)
print(f"Total de usuários processados: {len(lista_usuarios)}")
print(f"Sucessos: {len(usuarios_sucesso)}")
print(f"Erros: {len(usuarios_erro)}")
print(f"Taxa de sucesso: {len(usuarios_sucesso)/len(lista_usuarios)*100:.1f}%")

if usuarios_erro:
    print(f"\nUsuários com erro: {', '.join(usuarios_erro)}")

# Salvar resultados em variável global para acesso posterior
globals()['resultados_completos'] = resultados_usuarios

print("\n✅ Pipeline concluído! Resultados salvos em 'resultados_completos'")
print("Use 'resultados_completos[\"nome_usuario\"]' para acessar resultados específicos")


🚀 Iniciando pipeline de Machine Learning por usuário...
=== OBTENDO USUÁRIOS DO ARQUIVO ===
Usuários encontrados: ['usuario_00' 'usuario_01' 'usuario_02' 'usuario_03' 'usuario_04'
 'usuario_05' 'usuario_06' 'usuario_07' 'usuario_08' 'usuario_09'
 'usuario_10' 'usuario_11' 'usuario_12' 'usuario_13' 'usuario_14'
 'usuario_15' 'usuario_16']

🔄 Processando usuário 1/17: usuario_00
----------------------------------------
❌ Erro ao processar usuário usuario_00: name 'main' is not defined

🔄 Processando usuário 2/17: usuario_01
----------------------------------------
❌ Erro ao processar usuário usuario_01: name 'main' is not defined

🔄 Processando usuário 3/17: usuario_02
----------------------------------------
❌ Erro ao processar usuário usuario_02: name 'main' is not defined

🔄 Processando usuário 4/17: usuario_03
----------------------------------------
❌ Erro ao processar usuário usuario_03: name 'main' is not defined

🔄 Processando usuário 5/17: usuario_04
----------------------------